In [ ]:
import qsharp
import os, sys

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from util import plot

![](atom.excalidraw.svg)

In [ ]:
%%qsharp

import Std.Math.*;

operation PoloniumAtomAndObserver() : Result {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    Ry(PI() / 5.0, atom);

    // measurement simulates the observer
    let decay = M(atom);
    Reset(atom);
    decay
}

In [ ]:
results = qsharp.run("PoloniumAtomAndObserver()", shots=1000)
plot(results, title="Polonium Atom: Observed Decay Outcomes")

![](cat2.excalidraw.svg)

In [ ]:
%%qsharp

import Std.Math.*;

operation AtomCatAndObserver() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    Ry(PI() / 5.0, atom);

    // a pair of qubits pretend to be a cat
    // they are entangled with the atom
    use (cat_front, cat_back) = (Qubit(), Qubit());
    CNOT(atom, cat_front);
    CNOT(atom, cat_back);

    // measurement simulates the observer
    MResetEachZ([atom, cat_front, cat_back])
}

In [ ]:
results = qsharp.run("AtomCatAndObserver()", shots=1000)
plot(results, title="Atom and Cat Entanglement")

![](env2.excalidraw.svg)

In [13]:
%%qsharp

import Std.Math.*;

operation AtomCatObserverAndEnvironment() : Result[] {
    // qubit in a superposition pretends to be a polonium atom
    use atom = Qubit();
    Ry(PI() / 5.0, atom);

    // a pair of qubits pretend to be a cat
    // they are entangled with the atom
    use (cat_front, cat_back) = (Qubit(), Qubit());
    CNOT(atom, cat_front);
    CNOT(atom, cat_back);

    // 5 more qubits pretend to be the environment, also entangled
    mutable env_results = [Zero, size = 5];
    use env = Qubit[5];
    for i in 0..4 {
        CNOT(atom, env[i]);

        // simulate decoherence
        env_results w/= i <- MResetZ(env[i]); 
    }

    // measurement simulates the observer
    let atomAndCat = MResetEachZ([atom, cat_front, cat_back]);

    atomAndCat + env_results
}

In [16]:
results = qsharp.run("AtomCatObserverAndEnvironment()", shots=1000)

# note: we have 2^8 = 256 possible outcomes, so for clarity we hide the empty outcomes
plot(results, title="Atom, Cat, Observer and Environment Entanglement", hide_empty=True)